In [1]:
import json
import numpy as np
import pandas as pd
import os
from os import path
import shutil

In [2]:
annos_directory = "/Users/Admin/Documents/Study/DeepFashion2/train/annos/"
image_directory = "/Users/Admin/Documents/Study/DeepFashion2/train/image/"

In [18]:
len(os.listdir(annos_directory))


191961

In [ ]:
# map the categories to a binary namespace {'top', 'bottom'}
'''
Top:
----
short sleeve top - 1
long sleeve top - 2
short sleeve outwear - 3
long sleeve outwear - 4
vest - 5

Bottom:
-------
shorts - 7
trousers - 8
skirts - 9

'''

In [24]:
labeled_data = pd.DataFrame(columns=['image_name', 'category_id', 'category_name', 'bounding_box'])

# from every json dict, remove source and pair_id elements. The remaining dict items are the n item components

for filename in os.listdir(annos_directory):
    with(open(annos_directory+filename, "r")) as f:
        data = json.load(f)
        if(data['source'] != 'shop'):
            continue
        del data['source']
        del data['pair_id']
        if(len(data.items()) == 2):
            categoryids = []
            temp_df = pd.DataFrame(columns=['image_name', 'category_id', 'category_name', 'bounding_box'])
            for k,v in data.items():
                image_name = filename.split('.')[0]+'.jpg'
                category_id = v['category_id']
                category_name = v['category_name']
                bounding_box = v['bounding_box']
                if(category_id in [1,2,3,4,5]):
                    category_id = 1
                elif(category_id in [7,8,9]):
                    category_id = 2
                categoryids.append(category_id)
                d = {'image_name':image_name, 'category_id':category_id, 'category_name':category_name, 'bounding_box':bounding_box}
                temp_df = temp_df.append(pd.Series(d), ignore_index=True)
            if(1 in categoryids and 2 in categoryids): # contains both top and bottom
                labeled_data = labeled_data.append(temp_df, ignore_index=True)
                
                
                
len(labeled_data)            
#labeled_data.to_csv("/Users/Admin/Documents/Study/DeepFashion2/final/annos.csv", index=False)
            


165718

In [26]:
# Change category_name according to category_id
# Make bounding box list as separate columns

labeled_data.loc[labeled_data['category_id'] == 1, 'category_name'] = 'Top'
labeled_data.loc[labeled_data['category_id'] == 2, 'category_name'] = 'Bottom'



list

In [38]:
labeled_data[['xmin','ymin', 'xmax', 'ymax']] = pd.DataFrame(labeled_data.bounding_box.values.tolist(), index= labeled_data.index)

labeled_data.head()

,image_name,category_id,category_name,bounding_box,xmin,ymin,xmax,ymax
0,000011.jpg,2,Bottom,"[284, 548, 458, 851]",284,548,458,851
1,000011.jpg,1,Top,"[256, 393, 369, 608]",256,393,369,608
2,000012.jpg,2,Bottom,"[266, 380, 467, 511]",266,380,467,511
3,000012.jpg,1,Top,"[228, 232, 500, 493]",228,232,500,493
4,000013.jpg,2,Bottom,"[32, 481, 344, 701]",32,481,344,701


In [39]:
# get the image file names
len(set(labeled_data['image_name'].tolist()))

82859

In [40]:
# get those filenames for which the images are present

image_directory = "/Users/Admin/Documents/Study/DeepFashion2/train/image/"
final_images = []

for filename in set(labeled_data['image_name'].tolist()):
    source = image_directory+filename
    if(os.path.isfile(source)):
        final_images.append(filename)
        
len(final_images)

32980

In [41]:
# pick the top 1536 images
final_images_full = final_images[:1536]

len(final_images_full)

1536

In [45]:
# move the corresponding image files to a separate directory

image_directory = "/Users/Admin/Documents/Study/DeepFashion2/train/image/"
destination_path = "/Users/Admin/Documents/Study/DeepFashion2/final/images/"
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

for filename in final_images_full:
    source = image_directory+filename
    shutil.copy(source, destination_path)
    

In [58]:
# subset the dataframe to get the annotations for the 1536 images

labeled_data_subset = labeled_data

labeled_data_final = labeled_data_subset[labeled_data_subset['image_name'].isin(final_images_full)]

len(labeled_data_final)

3072

In [59]:
labeled_data_final.head()

,image_name,category_id,category_name,bounding_box,xmin,ymin,xmax,ymax
90,000087.jpg,2,Bottom,"[232, 412, 513, 696]",232,412,513,696
91,000087.jpg,1,Top,"[176, 2, 601, 489]",176,2,601,489
262,000374.jpg,1,Top,"[249, 186, 496, 482]",249,186,496,482
263,000374.jpg,2,Bottom,"[171, 407, 417, 833]",171,407,417,833
304,000402.jpg,1,Top,"[200, 0, 440, 84]",200,0,440,84


In [60]:
# add filepath to the image_name and drop bounding_box, category_id columns

#labeled_data_final['image_name'] = labeled_data_final['image_name'].apply(lambda x: "/Users/Admin/Documents/Study/DeepFashion2/final/images/"+x)

labeled_data_final = labeled_data_final.drop(['category_id', 'bounding_box'], axis=1)

labeled_data_final.head()


,image_name,category_name,xmin,ymin,xmax,ymax
90,000087.jpg,Bottom,232,412,513,696
91,000087.jpg,Top,176,2,601,489
262,000374.jpg,Top,249,186,496,482
263,000374.jpg,Bottom,171,407,417,833
304,000402.jpg,Top,200,0,440,84


In [61]:

labeled_data_final.to_csv("/Users/Admin/Documents/Study/DeepFashion2/final/annotations.csv", index=False)


In [62]:
## Split 1536 images to train, test
# Select the first 1280 images as train
train = final_images_full[:1280]

test = final_images_full[1280:]

len(train), len(test)

(1280, 256)

In [82]:
# move the corresponding image files to separate train and test directories

image_directory = "/Users/Admin/Documents/Study/DeepFashion2/final/images/"
destination_path_train = "/Users/Admin/Documents/Study/DeepFashion2/final/images/train/"
destination_path_test = "/Users/Admin/Documents/Study/DeepFashion2/final/images/test/"

if not os.path.exists(destination_path_train):
    os.makedirs(destination_path_train)

for filename in train:
    source = image_directory+filename
    shutil.move(source, destination_path_train)
    
if not os.path.exists(destination_path_test):
    os.makedirs(destination_path_test)
    
for filename in test:
    source = image_directory+filename
    shutil.move(source, destination_path_test)
    


In [63]:
train_data = labeled_data_final[labeled_data_final['image_name'].isin(train)]
test_data = labeled_data_final[labeled_data_final['image_name'].isin(test)]


In [64]:
train_data.head()

,image_name,category_name,xmin,ymin,xmax,ymax
262,000374.jpg,Top,249,186,496,482
263,000374.jpg,Bottom,171,407,417,833
304,000402.jpg,Top,200,0,440,84
305,000402.jpg,Bottom,186,27,464,432
434,000549.jpg,Top,162,0,477,156


In [65]:
test_data.head()

,image_name,category_name,xmin,ymin,xmax,ymax
90,000087.jpg,Bottom,232,412,513,696
91,000087.jpg,Top,176,2,601,489
872,001017.jpg,Bottom,12,802,557,1319
873,001017.jpg,Top,259,440,792,925
1072,001298.jpg,Top,235,245,431,499


In [ ]:
train_data.to_csv("/Users/Admin/Documents/Study/DeepFashion2/final/train.csv", index=False)
test_data.to_csv("/Users/Admin/Documents/Study/DeepFashion2/final/test.csv", index=False)


In [77]:
train_data['xmin'] = train_data['xmin'].apply(str)
train_data['ymin'] = train_data['ymin'].apply(str)
train_data['xmax'] = train_data['xmax'].apply(str)
train_data['ymax'] = train_data['ymax'].apply(str)

train_data.dtypes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

image_name       object
category_name    object
xmin             object
ymin             object
xmax             object
ymax             object
dtype: object

In [83]:
## Convert the annotations to text file required by faster r-cnn implementation

data = pd.DataFrame()
data['format'] = train_data['image_name']

# as the images are in train_images folder, add train_images before the image name
data['format'] = 'train/' + data['format']

# add xmin, ymin, xmax, ymax and class as per the format required

data['format'] = data['format'] + ',' + train_data['xmin'] + ',' + train_data['ymin'] + ',' + train_data['xmax'] + ',' + train_data['ymax'] + ',' + train_data['category_name']

data.to_csv('/Users/Admin/Documents/Study/DeepFashion2/final/annotate.txt', header=None, index=None, sep=' ')
